In [1]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

2025-06-16 18:42:04.286025: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750099324.780487      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750099324.914333      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
dataset_dir = "/kaggle/input/tsinghua-zip/tsinghua-zip-noteddycrd/low-resolution"

batch_size = 32
img_height = 224
img_width = 224
validation_split = 0.05
seed = 123

train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_dir,
    validation_split=validation_split,
    subset="training",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

val_test_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_dir,
    validation_split=validation_split,
    subset="validation",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

# num_val_test_batches = tf.data.experimental.cardinality(val_test_ds).numpy()
# num_test_batches = int(num_val_test_batches * 0.5) 
# val_ds = val_test_ds.skip(num_test_batches)
# test_ds = val_test_ds.take(num_test_batches)



Found 40083 files belonging to 128 classes.
Using 38079 files for training.


I0000 00:00:1750099387.747642      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1750099387.748477      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Found 40083 files belonging to 128 classes.
Using 2004 files for validation.


In [3]:
class_names = train_ds.class_names
num_classes = len(train_ds.class_names)

In [4]:
print(class_names)

['affenpinscher', 'afghan_hound', 'african_hunting_dog', 'airedale', 'american_staffordshire_terrier', 'appenzeller', 'australian_shepherd', 'australian_terrier', 'basenji', 'basset', 'beagle', 'bedlington_terrier', 'bernese_mountain_dog', 'bichon_frise', 'black_and_tan_coonhound', 'black_sable', 'blenheim_spaniel', 'bloodhound', 'bluetick', 'border_collie', 'border_terrier', 'borzoi', 'boston_bull', 'bouvier_des_flandres', 'boxer', 'brabancon_griffo', 'briard', 'brittany_spaniel', 'bull_mastiff', 'cairn', 'cane_carso', 'cardigan', 'chesapeake_bay_retriever', 'chihuahua', 'chinese_crested_dog', 'chow', 'clumber', 'cocker_spaniel', 'collie', 'curly_coated_retriever', 'dandie_dinmont', 'dhole', 'dingo', 'doberman', 'english_foxhound', 'english_setter', 'english_springer', 'entlebucher', 'eskimo_dog', 'fila_braziliero', 'flat_coated_retriever', 'french_bulldog', 'german_shepherd', 'german_short_haired_pointer', 'giant_schnauzer', 'golden_retriever', 'gordon_setter', 'great_dane', 'great_p

In [5]:
from tensorflow.keras.applications import EfficientNetV2L
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',    
    patience=10,            
    restore_best_weights=True  
)

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

base_model = EfficientNetV2L(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False 

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy',
            tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3, name='top_3_accuracy'),
            tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5, name='top_5_accuracy')]
)

history = model.fit(
    train_ds,
    validation_data=val_test_ds,
    epochs=10,            
    callbacks=[early_stopping]
)

473176280/473176280 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/10


I0000 00:00:1750099486.077014      69 service.cc:148] XLA service 0x7d353c140430 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1750099486.078657      69 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1750099486.078682      69 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1750099494.658066      69 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1750099542.904696      69 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1190/1190 ━━━━━━━━━━━━━━━━━━━━ 633s 424ms/step - accuracy: 0.7102 - loss: 1.3415 - top_3_accuracy: 0.8568 - top_5_accuracy: 0.8898 - val_accuracy: 0.8458 - val_loss: 0.5047 - val_top_3_accuracy: 0.9676 - val_top_5_accuracy: 0.9805
Epoch 2/10
1190/1190 ━━━━━━━━━━━━━━━━━━━━ 429s 361ms/step - accuracy: 0.8310 - loss: 0.5744 - top_3_accuracy: 0.9588 - top_5_accuracy: 0.9781 - val_accuracy: 0.8558 - val_loss: 0.4658 - val_top_3_accuracy: 0.9701 - val_top_5_accuracy: 0.9820
Epoch 3/10
1190/1190 ━━━━━━━━━━━━━━━━━━━━ 430s 361ms/step - accuracy: 0.8454 - loss: 0.5130 - top_3_accuracy: 0.9654 - top_5_accuracy: 0.9830 - val_accuracy: 0.8603 - val_loss: 0.4595 - val_top_3_accuracy: 0.9711 - val_top_5_accuracy: 0.9815
Epoch 4/10
1190/1190 ━━━━━━━━━━━━━━━━━━━━ 428s 359ms/step - accuracy: 0.8540 - loss: 0.4725 - top_3_accuracy: 0.9712 - top_5_accuracy: 0.9879 - val_accuracy: 0.8563 - val_loss: 0.4614 - val_top_3_accuracy: 0.9721 - val_top_5_accuracy: 0.9835
Epoch 5/10
1190/1190 ━━━━━━━━━━━━━━━━━━━━ 4

In [6]:
# model.evaluate(test_ds)

In [7]:
model.save('model.keras')

In [8]:
maltese_url = "https://teketrek.net/wp-content/uploads/2024/05/%D9%83%D9%84%D8%A8-%D8%A7%D9%84%D9%85%D8%A7%D9%84%D8%B7%D9%8A.webp"
maltese_path = tf.keras.utils.get_file('Maltese', origin=maltese_url)

97714/97714 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step


In [9]:
img = tf.keras.utils.load_img(
    maltese_path, target_size=(img_height, img_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

In [10]:
predictions = model.predict(img_array)
score = predictions[0]

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)


1/1 ━━━━━━━━━━━━━━━━━━━━ 29s 29s/step
This image most likely belongs to chinese_crested_dog with a 41.03 percent confidence.
